# **Global settings**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import rcParams
from matplotlib.pyplot import rc_context
import scanpy as sc
import scrublet as scr
import scvelo as scv
import seaborn as sns
import scipy.io
import os
import dotplot
import dotplot.utils
import math
import gseapy as gp
from gseapy.plot import barplot, dotplot, gseaplot
from gseapy.scipalette import SciPalette
from pylab import *
from matplotlib.colors import ListedColormap,LinearSegmentedColormap 

In [ ]:
sc.settings.set_figure_params(dpi=100, dpi_save=300, figsize=(5, 5))

In [ ]:
plt.set_cmap('viridis')

<Figure size 500x500 with 0 Axes>

In [ ]:
scv.set_figure_params()

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc')

# LDSC

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import decoupler as dc
import scanpy as sc
import os
import math
import csv

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC')
concat = sc.read_h5ad('/disk213/xieqq/JINHUA138.sc/adata_rowcounts.h5ad')
adata = sc.read_h5ad('/disk213/xieqq/JINHUA138.sc/adata_CellLineage.h5ad')
common_cells = set(concat.obs_names) & set(adata.obs_names) 
concat = concat[concat.obs_names.isin(common_cells), :]
adata = adata[adata.obs_names.isin(common_cells), :]
adata.layers['counts']=concat.X
my_sample_col = 'CellLineage'

Epithelial = sc.read_h5ad('/disk213/xieqq/JINHUA138.sc/Epithelial_CellType.h5ad')
adata = adata[adata.obs['CellLineage'].isin(['Epithelial'])]
adata.obs['CellType'] = Epithelial.obs['CellType'] 
my_sample_col = 'CellType'

In [ ]:
data1 = pd.read_csv('/disk213/xieqq/Software/pig_to_human_orthologs/pig_to_human_orthologs.csv')
data2 = pd.read_csv('/disk213/xieqq/Software/pig_to_human_orthologs/gProfiler_sscrofa.csv')
data = pd.merge(data1, data2[['initial_alias','converted_alias']], left_on='ensembl_gene_id', right_on='initial_alias')
name = pd.DataFrame({'gene_name': adata.var_names})
name1 = name[name['gene_name'].str.startswith('ENSSSCG')]
name1 = pd.merge(name1, data[['human_orthologs','ensembl_gene_id']], left_on='gene_name', right_on='ensembl_gene_id')
name2 = pd.merge(name, data[['human_orthologs','converted_alias']], left_on='gene_name', right_on='converted_alias')
name1 = name1.drop(name1.columns[-1], axis=1)
name2 = name2.drop(name2.columns[-1], axis=1)
name = pd.concat([name1, name2], axis=0)
name = name.dropna(subset=['human_orthologs'])
name = name.drop_duplicates(subset=['human_orthologs'], keep='first')
modified_names = []
for gene_name in adata.var_names:
    matches = name.loc[name['gene_name'] == gene_name, 'human_orthologs']
    if len(matches) > 0:
        modified_names.append(matches.values[0])
    else:
        modified_names.append(gene_name)
adata.var_names_make_unique()
adata.var_names = modified_names

In [ ]:
pdata = dc.get_pseudobulk(
    adata,
    sample_col=my_sample_col,  #CellLineage, CellType
    groups_col=None,
    layer='counts',
    mode='sum',
    min_cells=10,
    min_counts=1000
)

# Store raw counts in layers
pdata.layers['counts'] = pdata.X.copy()

# Normalize, scale and compute pca
sc.pp.normalize_total(pdata, target_sum=1e4)
sc.pp.log1p(pdata)
sc.pp.scale(pdata, max_value=10)
sc.tl.pca(pdata)
list = pdata.T.to_df()
list = list[list.index.str.contains("ENSG")]
#list.to_csv("raw_counts_scaled_CellLineage.csv", index=True)
list.to_csv("raw_counts_scaled_Epithelial.csv", index=True)

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC')
#adata = sc.read_h5ad('/disk213/xieqq/JINHUA138.sc/adata_CellLineage.h5ad')
#my_group = 'CellLineage'
adata = sc.read_h5ad('/disk213/xieqq/JINHUA138.sc/Epithelial_CellType.h5ad')
my_group = 'CellType'
adata.uns['log1p']["base"] = None

In [ ]:
data1 = pd.read_csv('/disk213/xieqq/Software/pig_to_human_orthologs/pig_to_human_orthologs.csv')
data2 = pd.read_csv('/disk213/xieqq/Software/pig_to_human_orthologs/gProfiler_sscrofa.csv')
data = pd.merge(data1, data2[['initial_alias','converted_alias']], left_on='ensembl_gene_id', right_on='initial_alias')
name = pd.DataFrame({'gene_name': adata.var_names})
name1 = name[name['gene_name'].str.startswith('ENSSSCG')]
name1 = pd.merge(name1, data[['human_orthologs','ensembl_gene_id']], left_on='gene_name', right_on='ensembl_gene_id')
name2 = pd.merge(name, data[['human_orthologs','converted_alias']], left_on='gene_name', right_on='converted_alias')
name1 = name1.drop(name1.columns[-1], axis=1)
name2 = name2.drop(name2.columns[-1], axis=1)
name = pd.concat([name1, name2], axis=0)
name = name.dropna(subset=['human_orthologs'])
name = name.drop_duplicates(subset=['human_orthologs'], keep='first')
modified_names = []
for gene_name in adata.var_names:
    matches = name.loc[name['gene_name'] == gene_name, 'human_orthologs']
    if len(matches) > 0: 
        modified_names.append(matches.values[0])
    else: 
        modified_names.append(gene_name)

adata.var_names_make_unique()
adata.var_names = modified_names

In [ ]:
sc.tl.rank_genes_groups(adata, groupby=my_group, method='wilcoxon', corr_method='benjamini-hochberg', n_genes=1000, key_added='DEG')
result = adata.uns['DEG']
groups = result['names'].dtype.names
marker = []
for group in groups:
    data = pd.DataFrame({'names': result['names'][group], 'scores': result['scores'][group], 'celltype':group})
    data = data[data['names'].str.contains("ENSG")]
    data = data.nlargest(500, 'scores')
    marker.append(data)

list = pd.concat(marker, axis=0)
#list.to_csv("DEG500_CellLineage.csv",index=False)
list.to_csv("DEG500_Epithelial.csv",index=False)

### R

In [ ]:
library(tidyverse)
library(data.table)
library(doParallel)
library(foreach)

In [ ]:
plink_dir<-"/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/1000G_EAS_Phase3_plink"
gene_coord<-"/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/all_gene.txt"
out_dir<-"/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/EAS_baseline_Epithelial" 
tseT_file<-"/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/DEG500_Epithelial.csv" 
dir.create(out_dir,showWarnings = F) 

In [ ]:
# TSG
tseT<-read.csv(tseT_file)
tsg<-list()
for(i in unique(tseT$celltype)){
  tsg[[i]]<-tseT[which(tseT$celltype==i),1]
}
tissues<-names(tsg)

# Coordinate GENE, CHR, START, and END
coord<-read.table(gene_coord,h=T)
gene_info<-data.frame(GENE=coord$gene_id,CHR=coord$chr,START=coord$start_pos,END=coord$end_pos)
write.table(gene_info,file=str_glue("{out_dir}/ENSSSCG_coord.txt"),
            row.names = F,quote = F,sep="\t")

# make annotation
make_annot<-function(tissue,out_dir,plink_dir){
  out_dir2<-str_glue("{out_dir}/{tissue}")
  dir.create(out_dir2,showWarnings = F)
  write.table(tsg[[tissue]],file=str_glue("{out_dir2}/tsg.txt"),row.names = F,col.names = F,quote = F)
  for(chr in 1:22){
    cmd<-str_glue("/disk212/xieqq/miniconda3/envs/ldsc/bin/python2 /disk213/xieqq/JINHUA138.sc/ldsc-master/make_annot.py --gene-set-file {out_dir2}/tsg.txt --gene-coord-file {out_dir}/ENSSSCG_coord.txt --windowsize 100000 --bimfile {plink_dir}/1000G.EAS.QC.{chr}.bim --annot-file {out_dir2}/baseline.{chr}.annot.gz")
    system(cmd)
  }
  return(NULL)
}

num_threads<-length(tissues)
registerDoParallel(num_threads)
results <- foreach(i = tissues, .combine = c) %dopar% {
  make_annot(i,out_dir=out_dir,plink_dir=plink_dir)
}

# combine annotation and calculate LDSC
ldsc_annot<-function(chr,plink_dir,out_dir){
  annots<-map(tissues,~{
    out_dir2<-str_glue("{out_dir}/{.x}")
    annot<-fread(str_glue("{out_dir2}/baseline.{chr}.annot.gz"))
    colnames(annot)[5]<-.x
    return(annot)
  })
  
  annot_all<-reduce(annots,merge)
  annot_all<-add_column(annot_all,1,.after="BP")
  colnames(annot_all)[5]<-"ALL"
  bim<-fread(str_glue("{plink_dir}/1000G.EAS.QC.{chr}.bim"))
  annot_all<-annot_all[match(bim$V2,annot_all$SNP),]
  fwrite(annot_all,file=str_glue("{out_dir}/baseline.{chr}.annot"),row.names=F,quote=F,sep="\t")
  
  cmd<-str_glue("gzip -f {out_dir}/baseline.{chr}.annot")
  system(cmd)
  cmd<-str_glue("/disk212/xieqq/miniconda3/envs/ldsc/bin/python2 /disk213/xieqq/JINHUA138.sc/ldsc-master/ldsc.py --l2 --bfile {plink_dir}/1000G.EAS.QC.{chr} --ld-wind-kb 500.0 --annot {out_dir}/baseline.{chr}.annot.gz --out {out_dir}/baseline.{chr}")
  system(cmd)
  
  return(NULL)
}

num_threads<-22
registerDoParallel(num_threads)
results <- foreach(i = 1:22, .combine = c) %dopar% {
  ldsc_annot(i,plink_dir=plink_dir,out_dir=out_dir)
}


# gsMap

In [ ]:
import scanpy as sc

In [ ]:
gsmap --version

In [ ]:
os.chdir('/disk213/xieqq/JINHUA138.sc/gsMap')

In [ ]:
adata = sc.read_h5ad("gsMap_example_data/ST/E16.5_E1S1.MOSTA.h5ad")  

In [ ]:
os.chdir('/disk213/xieqq/Yexw/gsMap')
priority_genes = pd.read_csv("/disk212/yexw/wgs/ASF/3_2/2_7_gene_score/1_top_genes.csv",index_col=None)
gene_list = priority_genes['Gene_Name'].tolist()
gene_list = [g.capitalize() for g in gene_list]
found_genes = set(gene_list) & set(adata.var_names)

sc.tl.score_genes(adata, found_genes, score_name='gene_signature_score')

In [ ]:
sc.pl.spatial(adata,img_key="hires",color=['gene_signature_score'],spot_size=1.5,save='_E16.5_E1S1.MOSTA_priority_genes.pdf')

In [ ]:
sc.pl.spatial(adata,img_key="hires",color="annotation",spot_size=1.5,save='_E16.5_E1S1.MOSTA.pdf')

In [ ]:
adata.obs['CellSelect'] = adata.obs['annotation'].apply(lambda x: 'GI tract' if x == 'GI tract' else 'NA')

In [ ]:
sc.pl.spatial(adata,img_key="hires",color="CellSelect",spot_size=1.5)

## Quick Mode

In [ ]:
wget https://portals.broadinstitute.org/collaboration/giant/images/4/4e/GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_ALL.gz
gzip -d GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_ALL.gz

gsmap format_sumstats \
--sumstats 'GIANT_HEIGHT_YENGO_2022_GWAS_SUMMARY_STATS_ALL' \
--out 'HEIGHT'

zcat HEIGHT.sumstats.gz | head -n 5

In [ ]:
wget https://yanglab.westlake.edu.cn/data/gsMap/gsMap_resource.tar.gz
tar -xvzf gsMap_resource.tar.gz

In [ ]:
wget https://yanglab.westlake.edu.cn/data/gsMap/gsMap_example_data.tar.gz
tar -xvzf gsMap_example_data.tar.gz

In [ ]:
adata = sc.read_h5ad("gsMap_example_data/human_intestine/Spatial_Human_majority_voting.h5ad")  

In [ ]:
adata = sc.read_h5ad("gsMap_example_data/ST/E16.5_E1S1.MOSTA.h5ad")  

In [ ]:
# 1. find latent representations (optional)
gsmap run_find_latent_representations \
    --workdir './example/Mouse_Embryo' \
    --sample_name 'E16.5_E1S1.MOSTA' \
    --input_hdf5_path 'gsMap_example_data/ST/E16.5_E1S1.MOSTA.h5ad' \
    --annotation 'annotation' \
    --data_layer 'count'

In [ ]:
# 2. generate gene specificity scores
gsmap run_latent_to_gene \
    --workdir './example/Mouse_Embryo' \
    --sample_name 'E16.5_E1S1.MOSTA' \
    --annotation 'annotation' \
    --latent_representation 'latent_GVAE' \
    --num_neighbour 51 \
    --num_neighbour_spatial 201 \
    --homolog_file 'gsMap_resource/homologs/mouse_human_homologs.txt'

In [ ]:
# 3. generate ldscore
for CHROM in {1..22}
do
    gsmap run_generate_ldscore \
        --workdir './example/Mouse_Embryo' \
        --sample_name 'E16.5_E1S1.MOSTA' \
        --chrom $CHROM \
        --bfile_root '/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/1000G_EAS_Phase3_plink/1000G.EAS.QC' \
        --keep_snp_root 'gsMap_resource/LDSC_resource/hapmap3_snps/EAS_hm3' \
        --gtf_annotation_file 'gsMap_resource/genome_annotation/gtf/gencode.v39lift37.annotation.gtf' \
        --gene_window_size 50000
done

In [ ]:
# 4. spatial ldsc
gsmap run_spatial_ldsc \
    --workdir './example/Mouse_Embryo' \
    --sample_name 'E16.5_E1S1.MOSTA' \
    --trait_name 'ApolipoproteinB_UKBiobank' \
    --sumstats_file '/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/all_sumstats_pass/LDSCORE_all_sumstats_UKB_460K.biochemistry_ApolipoproteinB.sumstats.gz' \
    --w_file '/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/1000G_EAS_Phase3_weights_hm3_no_MHC/weights.EAS.hm3_noMHC.' \
    --num_processes 4

In [ ]:
# 5. cauchy combination (optional)
gsmap run_cauchy_combination \
    --workdir './example/Mouse_Embryo' \
    --sample_name 'E16.5_E1S1.MOSTA' \
    --trait_name 'Cholesterol_UKBiobank' \
    --annotation 'annotation'

In [ ]:
# 6. report generation (optional)
gsmap run_report \
    --workdir './example/Mouse_Embryo' \
    --sample_name 'E16.5_E1S1.MOSTA' \
    --trait_name 'TotalBilirubin_UKBiobank' \
    --annotation 'annotation' \ 
    --sumstats_file '/disk213/xieqq/JINHUA138/Single_cell_analysis.10.LDSC/data/all_sumstats_pass/LDSCORE_all_sumstats_UKB_460K.biochemistry_TotalBilirubin.sumstats.gz' \
    --top_corr_genes 50